## Data Acquisition

In [1]:
!pip install geemap google-cloud-storage rasterio matplotlib > /dev/null 2>&1

In [2]:
import ee
import geemap
import os
import pandas as pd
from google.colab import drive
import rasterio
import matplotlib.pyplot as plt


drive.mount('/content/drive')
work_dir = '/content/drive/MyDrive/UHI-Detection-Analysis/data/raw/'
os.makedirs(work_dir, exist_ok=True)


Mounted at /content/drive


In [3]:
# GEE kimlik doğrulama
ee.Authenticate()
ee.Initialize(project='manifest-pride-258211')

**MODIS for selecting hotest days through 10 year**

In [ ]:
# In[2]:
# --- Konfigürasyon ve Gerekli Tanımlamalar ---

# Çalışılacak alan (Area of Interest - AOI)
place_name = "Hamburg, Germany"

# OSM verileri için kullanılacak etiketler
# Projenin ihtiyacına göre bu etiketler zenginleştirilebilir.
tags = {
    "building": True,
    "highway": True,
    "leisure": ["park", "garden", "playground"],
    "landuse": ["forest", "grass", "greenfield"],
    "natural": ["water", "wetland"]
}

# Çıktıların kaydedileceği klasör
PROCESSED_DIR = '../data/processed/'
os.makedirs(PROCESSED_DIR, exist_ok=True)
print(f"İşlenmiş veriler '{PROCESSED_DIR}' klasörüne kaydedilecek.")

# Referans Raster: Tüm çıktıların bu raster'ın boyut, projeksiyon (CRS) ve
# çözünürlüğüne hizalanması çok önemlidir. Bu, katmanların üst üste
# tam oturmasını sağlar. Genellikle 10m çözünürlüklü bir Sentinel-2 bandı kullanılır.
# LÜTFEN BU YOLU KENDİ PROJENİZDEKİ BİR SENTINEL-2 DOSYASININ YOLU İLE GÜNCELLEYİN!
REFERENCE_RASTER_PATH = '../data/raw/sentinel2/B04_10m.jp2' # Örnek: Kırmızı bant (10m)

if not os.path.exists(REFERENCE_RASTER_PATH):
    raise FileNotFoundError(f"Referans raster dosyası bulunamadı: {REFERENCE_RASTER_PATH}. Lütfen bu yolu projenizdeki 10m'lik bir Sentinel-2 bandı ile güncelleyin.")

print(f"Referans raster olarak '{REFERENCE_RASTER_PATH}' kullanılacak.")

In [ ]:
def get_modis_hottest_days(start_year=2014, end_year=2024):

    """
    Finds the hottest cloud-free summer days (May 15 – Sep 15) for Hamburg using MODIS LST data.
    Returns a DataFrame with date, LST in Kelvin, and Celsius.
    """

    # Create a 5 km buffer around Hamburg coordinates
    hamburg = ee.Geometry.Point(9.9937, 53.5511).buffer(5000)
    results = {}

    for year in range(start_year, end_year + 1):
        print(f"\nProcessing year {year}...")

        try:
            # Load MODIS LST Day 1km collection
            modis = ee.ImageCollection('MODIS/061/MOD11A1') \
                .filterBounds(hamburg) \
                .filterDate(f'{year}-05-15', f'{year}-09-15')
                # .filter(ee.Filter.lt('QC_Day', 2))  # Optional: strict quality filter

            # Function to extract mean LST and acquisition date
            def compute_lst(img):
                mean_lst = img.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=hamburg,
                    scale=1000
                ).get('LST_Day_1km')
                return ee.Feature(None, {
                    'lst': mean_lst,
                    'date': img.date().format('YYYY-MM-dd')
                })

            # Map function over collection, and filter out null results
            lst_features = modis.map(compute_lst).filter(
                ee.Filter.notNull(['lst'])
            )

            # Check if any valid images remain
            if lst_features.size().getInfo() == 0:
                print("No valid LST data available, skipping.")
                continue

            # Sort by LST descending and pick the hottest
            hottest = ee.Feature(lst_features.sort('lst', False).first())

            date = hottest.get('date').getInfo()
            lst_kelvin = hottest.get('lst').getInfo()

            # Some years might return null
            if lst_kelvin is None:
                print("No temperature value found, skipping.")
                continue

            lst_celsius = lst_kelvin * 0.02 - 273.15

            results[year] = {
                'date': date,
                'lst_kelvin': lst_kelvin,
                'lst_celsius': round(lst_celsius, 2)
            }

            print(f"Hottest day: {date} | LST: {lst_celsius:.2f} °C")

        except Exception as e:
            print(f"Error: {str(e)}")
            continue

    return pd.DataFrame.from_dict(results, orient='index')

print("\Extracting MODIS data...")
df = get_modis_hottest_days()
print("\SResults:")
print(df)

\Extracting MODIS data...

Processing year 2014...
Hottest day: 2014-07-04 | LST: 34.83 °C

Processing year 2015...
Hottest day: 2015-07-05 | LST: 38.17 °C

Processing year 2016...
Hottest day: 2016-06-05 | LST: 35.11 °C

Processing year 2017...
Hottest day: 2017-05-27 | LST: 32.62 °C

Processing year 2018...
Hottest day: 2018-07-27 | LST: 37.34 °C

Processing year 2019...
Hottest day: 2019-06-30 | LST: 39.26 °C

Processing year 2020...
Hottest day: 2020-06-27 | LST: 36.38 °C

Processing year 2021...
Hottest day: 2021-06-17 | LST: 37.72 °C

Processing year 2022...
Hottest day: 2022-07-20 | LST: 39.18 °C

Processing year 2023...
Hottest day: 2023-07-15 | LST: 34.22 °C

Processing year 2024...
Hottest day: 2024-06-25 | LST: 33.50 °C
\SResults:
            date    lst_kelvin  lst_celsius
2014  2014-07-04  15399.234460        34.83
2015  2015-07-05  15566.021583        38.17
2016  2016-06-05  15413.236373        35.11
2017  2017-05-27  15288.740653        32.62
2018  2018-07-27  15524.5183

**Collecting Landsat 8 images based on hottest days and calculating LTS**

In [ ]:
# Hamburg koordinatları
hamburg = ee.Geometry.Point(9.99, 53.55)

def get_landsat_data(year, max_cloud=20):
    """
    Belirtilen yıl için en uygun Landsat 8 görüntüsünü bulur.
    Mantığı iyileştirildi ve sıralama hatası düzeltildi.
    """
    target_date = ee.Date(df.loc[year, 'date'])
    image_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(hamburg)

    # 1. Tam tarihte ara (+/- 1 gün, daha esnek)
    landsat = image_collection \
        .filterDate(target_date.advance(-1, 'day'), target_date.advance(1, 'day')) \
        .filter(ee.Filter.lt('CLOUD_COVER', max_cloud))

    # 2. Yoksa +/- 10 gün ara ve en az bulutluyu al
    if landsat.size().getInfo() == 0:
        print(f"{year} için +/- 1 günlük ve <{max_cloud}% bulutlu görüntü bulunamadı. Arama aralığı genişletiliyor...")
        landsat = image_collection \
            .filterDate(target_date.advance(-10, 'day'), target_date.advance(10, 'day')) \
            .filter(ee.Filter.lt('CLOUD_COVER', max_cloud)) \
            .sort('CLOUD_COVER') # En az bulutlu olanı başa getir

    # 3. Hâlâ yoksa bulut filtresini kaldır, en az bulutluyu bul ve maske uygula
    if landsat.size().getInfo() == 0:
        print(f"{year} için KRİTİK UYARI: Düşük bulutlu görüntü bulunamadı. QA bandı ile maskelenecek en iyi aday aranıyor...")
        landsat = image_collection \
            .filterDate(target_date.advance(-15, 'day'), target_date.advance(15, 'day')) \
            .sort('CLOUD_COVER') # En az bulutlu olanı bul (belki %50, belki %80 ama en iyisi bu)

    # Eğer herhangi bir aşamada görüntü bulunduysa, ilkini (en iyisini) al
    image = ee.Image(landsat.first())

    # Görüntü var mı diye kontrol et
    # .size() > 0 kontrolü sonrası .first() boş koleksiyonda hata vermemesi için
    # image'ı ee.Image() ile sarmak, varolmayan bir durumda bile bir nesne oluşturur.
    # Bu nesnenin özelliklerini sorgulayarak var olup olmadığını anlarız.
    if image.getInfo() is None:
        print(f"UYARI: {year} yılı için arama aralıklarında HİÇBİR görüntü bulunamadı.")
        return None

    # Eğer 3. aşamaya düştüysek, yani CLOUD_COVER filtresi uygulanmadıysa maskele
    if image.get('CLOUD_COVER').getInfo() >= max_cloud:
        print(f"{year} için bulunan en iyi görüntünün bulut oranı (%{image.get('CLOUD_COVER').getInfo():.2f}) yüksek. QA maskesi uygulanıyor.")
        # Cloud maskeleme fonksiyonu
        def mask_clouds(img):
            # Bulutlar için 3. biti ve bulut gölgesi için 4. biti kontrol et
            qa = img.select('QA_PIXEL')
            cloud_bit_mask = 1 << 3
            cloud_shadow_bit_mask = 1 << 4
            # Her ikisinin de olmadığı pikselleri seç
            mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0))
            return img.updateMask(mask)

        image = mask_clouds(image)

    return image


# LST hesaplama fonksiyonu (değişiklik yok)
def calculate_lst(image):
    # Fonksiyona gelen argümanın geçerli bir ee.Image olup olmadığını kontrol et
    if image is None:
        return None

    lst = image.expression(
        '(TIRS1 * 0.00341802 + 149.0) - 273.15',  # Kelvin'den Celsius'a
        {'TIRS1': image.select('ST_B10')}
    ).rename('LST')
    return image.addBands(lst) # LST bandını orijinal görüntüye ekle

# --- Görüntüleri al ve LST hesapla ---
years = df.index.tolist()
lst_images = {}

for year in years:
    print(f"\n--- {year} yılı işleniyor ---")
    landsat_image = get_landsat_data(year)

    if landsat_image:
        lst_image = calculate_lst(landsat_image)
        lst_images[year] = lst_image
        print(f"{year} için LST başarıyla hesaplandı.")
    else:
        print(f"{year} için LST hesaplanamadı çünkü uygun görüntü bulunamadı.")

# Örnek: 2024 LST görüntüsünü kontrol et
# if 2024 in lst_images:
#     print("\n2024 LST Görüntü Bilgisi:", lst_images[2024].getInfo())

**Visualize 10 years LTS data for control**

In [ ]:
# Interactive harita oluştur
Map = geemap.Map(center=[53.55, 9.99], zoom=12)

# Görselleştirme parametreleri (tek bant için)
vis_params = {
    'min': 20,  # Min LST (°C)
    'max': 40,  # Max LST (°C)
    'palette': ['blue', 'green', 'yellow', 'red']  # Veya 'inferno'
}

# lst_images sözlüğündeki her yıl için katman ekle
for year, lst_image in lst_images.items():
    try:
        # Görüntünün yalnızca LST bandını seç
        lst_single_band = lst_image.select('LST')
        # Haritaya katman ekle
        Map.addLayer(lst_single_band, vis_params, f'LST {year}')
    except Exception as e:
        print(f"{year} için hata: {str(e)}")

# Layer kontrol paneli ekle
Map.addLayerControl()

# Haritayı göster
Map

**Adding Time Slider to 10 years LTS**


**Export LTS as GeoTIF**

In [ ]:
# GeoTIFF olarak export etme fonksiyonu (GÜNCEL)
def export_to_drive(image, name, folder):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=name,
        folder=folder.replace('/content/drive/MyDrive/', ''),  # GEE için göreli yol
        fileNamePrefix=name,
        scale=30,
        region=hamburg.buffer(5000).bounds(),
        fileFormat='GeoTIFF'
    )
    task.start()
    return task

work_dir = 'raw'

# Drive'a kaydet (work_dir kullanarak)
export_to_drive(lst_2015, 'LST_2015_Hamburg', work_dir)
export_to_drive(lst_2024, 'LST_2024_Hamburg', work_dir)

# Task'lerin tamamlanmasını bekle
import time
while True:
    tasks = ee.batch.Task.list()
    if all(task.status()['state'] in ('COMPLETED', 'FAILED') for task in tasks):
        break
    time.sleep(10)
print(f"Export işlemleri tamamlandı!")